## Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_predict

from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import BaggingClassifier
from sklearn.svm import SVC
from sklearn.ensemble import StackingClassifier

from sklearn.linear_model import Perceptron
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.neural_network import MLPClassifier
from sklearn.cluster import KMeans
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.cluster import AgglomerativeClustering


## Train-Test Sample Dataset loading

In [2]:
dfSample0 = pd.read_csv(r'D:\Project Phase II\Dataset\dfSample0.csv',encoding='cp1252')
dfSample1 = pd.read_csv(r'D:\Project Phase II\Dataset\dfSample1.csv',encoding='cp1252')

dfSample = pd.concat([dfSample0, dfSample1])
dfSample = dfSample.sample(frac=1, random_state=42).reset_index(drop=True)

#Split the dataset into features and labels
XSample = dfSample.drop('Label', axis=1)
ySample = dfSample['Label']



C:\Users\sathi\AppData\Local\Temp\ipykernel_17096\2929915898.py:1: DtypeWarning: Columns (1,3) have mixed types. Specify dtype option on import or set low_memory=False.
  dfSample0 = pd.read_csv(r'D:\Project Phase II\Dataset\dfSample0.csv',encoding='cp1252')


In [3]:
#Display all columns
XSample.columns

Index(['srcip', 'sport', 'dstip', 'dsport', 'proto', 'state', 'dur', 'sbytes',
       'dbytes', 'sttl', 'dttl', 'sloss', 'dloss', 'service', 'Sload', 'Dload',
       'Spkts', 'Dpkts', 'swin', 'dwin', 'stcpb', 'dtcpb', 'smeansz',
       'dmeansz', 'trans_depth', 'res_bdy_len', 'Sjit', 'Djit', 'Stime',
       'Ltime', 'Sintpkt', 'Dintpkt', 'tcprtt', 'synack', 'ackdat',
       'ct_state_ttl', 'ct_flw_http_mthd', 'is_ftp_login', 'ct_ftp_cmd',
       'ct_srv_src', 'ct_srv_dst', 'ct_dst_ltm', 'ct_src_ ltm',
       'ct_src_dport_ltm', 'ct_dst_sport_ltm', 'ct_dst_src_ltm', 'attack_cat'],
      dtype='object')

Select Features to Drop

In [ ]:
featureset = []
todrop = ['srcip','dstip']

Train-test Split

In [ ]:
#Split the dataset into training and testing sets
XSample_train, XSample_test, ySample_train, ySample_test = train_test_split(XSample, ySample, test_size=0.2, random_state=42)